# SDS 2023 Forecasting & Meta Learning Workshop

In this notebook, we will get our hands dirty with Darts. We will do the following things:

* **Part 1:** Forecasting passenger counts series for 300 airlines (`air` dataset). We will train one model per series.
* **Part 2:** Using "global" models - i.e., models trained on all 300 series simultaneously. Here we split every timeseries into data from the trainset and data from the testset.
* **Part 3:** We will try some *meta learning*, and see what happens if we train some global models on one (big) dataset (`m4` dataset) and use them on another dataset. Compared to part 2, m4 is the trainset and m3 will be our testset.
* **Part 4:** We will reuse our pre-trained model(s) of Part 3 on another new dataset (`m3` dataset) and see how it compares to models specifically trained on this dataset.

## Part 0: Setup (No code to write - execute only)
First, we need to install the right libraries and make the right imports. For the deep learning models, it will help to use a GPU runtime. To get a GPU instance, click on the "RAM/Disk" info bars on the upper right, select "Change runtime type" and choose a GPU as hardware accelerator. The following command will show you the GPU available (if any). If there's no GPU available, you can still go ahead and work on CPU.

In [ ]:
# You can run this command to see if there's a GPU:
!nvidia-smi

In [ ]:
!pip install -q darts matplotlib seaborn tqdm sklearn

In [ ]:
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import os
import time
import random
import pandas as pd
import pickle
import numpy as np
import requests
import zipfile
import tqdm.notebook as tq
from datetime import datetime
import torch
from torch import nn
from typing import List, Tuple, Dict
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler
from sklearn.linear_model import Ridge
import matplotlib.pyplot as plt
import seaborn as sns
from pytorch_lightning.callbacks import Callback, EarlyStopping

from darts import TimeSeries
from darts.utils.losses import SmapeLoss
from darts.dataprocessing.transformers import Scaler, MissingValuesFiller
from darts.metrics import smape, mase, mape
from darts.utils.data import HorizonBasedDataset
from darts.utils.utils import SeasonalityMode, TrendMode, ModelMode
from darts.models import (
    NaiveSeasonal, NBEATSModel, ExponentialSmoothing,
    TCNModel, RegressionModel, LinearRegressionModel,
    LightGBMModel, ARIMA, Theta, KalmanForecaster
)

We define the forecast horizon here - for all of the (monthly) time series used in this notebook, we'll be interested in forecasting 18 months in advance. We pick 18 months as this is what is used in the M3/M4 competitions for monthly series.

In [ ]:
HORIZON = 18

### Datasets loading methods
Here, we define some helper methods to load the three datasets we'll be playing with: `air`, `m3` and `m4`.

All the methods below return two list of `TimeSeries`: one list of training series and one list of "test" series (of length `HORIZON`).

For convenience, all the series are already scaled here, by multiplying each of them by a constant so that the largest value is 1. Such scaling is necessary for many models to work correctly (esp. deep learning models). It does not affect the sMAPE values, so we can evaluate the accuracy of our algorithms on the scaled series. In a real application, we would have to keep the Darts `Scaler` objects somewhere in order to inverse-scale the forecasts (more in [this section of the documentation](https://unit8co.github.io/darts/generated_api/darts.dataprocessing.transformers.scaler.html)).

In [ ]:
# Execute this cell once to download all three datasets
!curl -L https://github.com/unit8co/sds2023-forecating-and-meta-learning/blob/main/data/m3_dataset.xls\?raw\=true -o m3_dataset.xls
!curl -L https://github.com/unit8co/sds2023-forecating-and-meta-learning/blob/main/data/passengers_per_carrier.csv\?raw\=true -o passengers_per_carrier.csv
!curl -L https://github.com/unit8co/sds2023-forecating-and-meta-learning/blob/main/data/m4_monthly_scaled.pkl\?raw\=true -o m4_monthly_scaled.pkl

In [ ]:
def load_m3() -> Tuple[List[TimeSeries], List[TimeSeries]]:
    print('building M3 TimeSeries...')

    # Read DataFrame
    df_m3 = (pd.read_excel('m3_dataset.xls', 'M3Month'))

    # Build TimeSeries
    m3_series = []
    for row in tq.tqdm(df_m3.iterrows(), position=0, leave=True):
        s = row[1]
        start_year = int(s['Starting Year'])
        start_month = int(s['Starting Month'])
        values_series = s[6:].dropna()
        if start_month == 0:
            continue

        start_date = datetime(year=start_year, month=start_month, day=1)
        time_axis = pd.date_range(start_date, periods=len(values_series))
        series = TimeSeries.from_times_and_values(time_axis, values_series.values).astype(np.float32)
        m3_series.append(series)

    print('\nThere are {} monthly series in the M3 dataset'.format(len(m3_series)))

    # Split train/test
    print('splitting train/test...')
    m3_train = [s[:-HORIZON] for s in m3_series]
    m3_test = [s[-HORIZON:] for s in m3_series]

    # Scale so that the largest value is 1
    print('scaling...')
    scaler_m3 = Scaler(scaler=MaxAbsScaler())
    m3_train_scaled: List[TimeSeries] = scaler_m3.fit_transform(m3_train)
    m3_test_scaled: List[TimeSeries] = scaler_m3.transform(m3_test)

    print('done. There are {} series, with average training length {}'.format(
        len(m3_train_scaled), np.mean([len(s) for s in m3_train_scaled])
    ))
    return m3_train_scaled, m3_test_scaled

def load_air() -> Tuple[List[TimeSeries], List[TimeSeries]]:
    df_per_carrier = pd.read_csv("/content/passengers_per_carrier.csv", dtype={"passengers": np.float32})
    df_per_carrier["month"] = df_per_carrier["month"].apply(pd.to_datetime)
    # Passenger filtering
    carriers_total_passenger = df_per_carrier.groupby("carrier_name").agg(total_passengers = ("passengers", "sum"))
    CARRIERS_TO_KEEP = carriers_total_passenger.query("total_passengers > 100000").index.values.tolist()
    df_per_carrier_filt = (
        df_per_carrier
        .query("carrier_name in @CARRIERS_TO_KEEP")
    )
    df_per_carrier_filt = df_per_carrier_filt.loc[:, ["carrier_name", "passengers", "month"]]
    min_len = 60
    air_data = TimeSeries.from_group_dataframe(df_per_carrier_filt, group_cols="carrier_name", value_cols="passengers", time_col="month", freq="MS")
    # Filtering
    air_data = [a for a in air_data if len(a) > min_len and a.min(axis=0).values() > 0]
    # Interpolating
    transformer = MissingValuesFiller()
    air_data = [transformer.transform(a) for a in air_data]
    # Train test
    air_train, air_test = [a[:-HORIZON] for a in air_data], [a[-HORIZON:] for a in air_data]
    # Rescaling between 0 and 1
    scaler_air = Scaler(scaler=MaxAbsScaler())
    air_train = scaler_air.fit_transform(air_train)
    air_test = scaler_air.transform(air_test)
    print('done. There are {} series, with average training length {}'.format(
          len(air_train), np.mean([len(s) for s in air_train])
      ))
    return air_train, air_test

def load_m4() -> Tuple[List[TimeSeries], List[TimeSeries]]:
    # load TimeSeries - the splitting and scaling has already been done
    print('loading M4 TimeSeries...')
    with open('m4_monthly_scaled.pkl', 'rb') as f:
        m4_series = pickle.load(f)
    m4_train_scaled, m4_test_scaled = zip(*m4_series)

    print('done. There are {} series, with average training length {}'.format(
        len(m4_train_scaled), np.mean([len(s) for s in m4_train_scaled])
    ))
    return m4_train_scaled, m4_test_scaled

Finally, we define a handy function to tell us how good a bunch of forecasted series are:

In [ ]:
def eval_forecasts(pred_series: List[TimeSeries],
                   test_series: List[TimeSeries]) -> List[float]:

    print('computing sMAPEs...')
    smapes = smape(test_series, pred_series)
    mean, std = np.mean(smapes), np.std(smapes)
    print('Avg sMAPE: %.3f +- %.3f' % (mean, std))
    plt.figure(figsize=(4,4), dpi=144)
    plt.hist(smapes, bins=50)
    plt.ylabel('Count')
    plt.xlabel('sMAPE')
    plt.show()
    plt.close()
    return smapes

## Part 1: Local models on the `air` dataset

The `air` dataset shows the number of air passengers that flew in or out of the USA per carrier (or airline company) from the year 2000 until 2019.

**Your turn:** First, you can load the train and test series by calling `load_air()` function that we have defined above.

In [ ]:
air_train, air_test = ...

It's a good idea to start by visualising a few of the series to get a sense of what they look like. We can plot a series by calling `series.plot()`.

In [ ]:
for i in [1, 20, 50, 100, 125]:
    plt.figure(figsize=(4,4), dpi=144)
    air_train[i].plot(label=air_train[i].static_covariates.loc["passengers", "carrier_name"])
    plt.ylabel('Passengers')
    plt.xlabel('Time')
    plt.show()
    plt.close()

We can see that most series look quite different, and they even have different time axes.

**Question:** What is the smallest series available?

### Forecasting with a first (very) naive model

We can now try a first forecasting model on this dataset. As a first step, it is usually a good practice to see how a (very) naive model blindly repeating the last value of the training series performs. This can be done in Darts using a [NaiveSeasonal](https://unit8co.github.io/darts/generated_api/darts.models.forecasting.baselines.html#darts.models.forecasting.baselines.NaiveSeasonal) model.

**Your turn:** Try forecasting a few time series of the air dataset using the `NaiveSeasonal` model. You can use the function `eval_forecasts` defined above to evaluate the forecasts.

### A useful function to evaluate models

Below, we write a small function that will make our life easier for quickly trying and comparing different local models. We loop through each serie, fit a model and then evaluate on our test dataset.

> ⚠️ Please note `tq.tqdm` is optional and is only there to help display the training progress (as you will see, depending on the model, it can take some time when training 300+ time series)


In [ ]:
def eval_local_model(train_series: List[TimeSeries],
                     test_series: List[TimeSeries],
                     model_cls,
                     **kwargs) -> Tuple[List[float], float]:
    preds = []
    start_time = time.time()
    for series in tq.tqdm(train_series):
        model = model_cls(**kwargs)
        model.fit(series)
        pred = model.predict(n=HORIZON)
        preds.append(pred)
    elapsed_time = time.time() - start_time
    smapes = eval_forecasts(preds, test_series)
    return smapes, elapsed_time

We can apply this function to fit all the time series of the dataset with the `NaiveSeasonal` model.

In [ ]:
naive_seasonal_last_smapes, naive_seasonal_last_elapsed_time  = eval_local_model(air_train, air_test, NaiveSeasonal, K=1)

So the most naive model gives us an average sMAPE of about 26.42.

**Your turn:** Can we do better with a "less naive" model exploiting the fact that most monthly series have a seasonality of 12?

All of the Darts forecasting models can be trained and used in the same way!
So we invite you to go over the [list of models in the API documentation](https://unit8co.github.io/darts/generated_api/darts.models.forecasting.html) and try a few more models. Here are some suggestions:

* `ExponentialSmoothing`
* `Theta`
* `ARIMA` - but the default parameters probably won't do very well. Using `p=12`, `d=1`, `q=0` might be a good start.
* ... Your ideas here!

We recommend that you keep track of the SMAPEs and elapsed times for each model. Later we will use these values for quickly comparing models. Some models will take longer than others to run. Don't hesitate to interrupt the execution or run only on a subset of series.

**Your turn:** Try to get the lowest possible errors with some other models of your choice.

In [ ]:
# model_X_smapes, model_X_elapsed_time = eval_local_model(air_train, air_test, ModelX, **hyper_params_for_model_X)

### Comparing models

Below, we define a couple of functions that we will use to obtain an overview of the SMAPEs and time required to obtain the predictions.

In [ ]:
def smapes_boxplot(method_to_smapes: Dict[str, List[float]], title: str):
  method_names = []
  smapes = []
  for curr_method_name, curr_smapes in method_to_smapes.items():
    method_names += [curr_method_name] * len(curr_smapes)
    smapes += curr_smapes
  smapes_df = pd.DataFrame({'Method': method_names, 'sMAPE': smapes})
  plt.figure(figsize=(7,4), dpi=144)
  ax = sns.boxplot(x="Method", y="sMAPE", data=smapes_df)
  ax.grid(False)
  # Display median score on each box
  medians = smapes_df.groupby(['Method'])['sMAPE'].median().round(decimals=2)
  vertical_offset = smapes_df['sMAPE'].median() * 0.1
  for xtick, name in enumerate(method_to_smapes.keys()):
    ax.text(xtick, medians[name] + vertical_offset, medians[name],
                  horizontalalignment='center', size='x-small', color='w', weight='semibold')
  plt.xticks(rotation=90)
  plt.title(title)
  plt.show()
  plt.close()

def elapsed_time_barplot(method_to_elapsed_times: Dict[str, float], title: str):
  elapsed_times_df = pd.DataFrame({'Method': method_to_elapsed_times.keys(), 'Elapsed time [s]': method_to_elapsed_times.values()})
  ax = plt.figure(figsize=(7,4), dpi=144)
  sns.barplot(x="Method", y="Elapsed time [s]", data=elapsed_times_df)
  plt.xticks(rotation=90)
  plt.title(title)
  plt.show()
  plt.close()

**Your turn:** We are now ready to visualise our models. Fill in the cells below to call `smapes_boxplot()` and `elpased_time_barplot()` with the right arguments.

In [ ]:
smapes = {
    'NaiveSeasonal last': naive_seasonal_last_smapes,
    # 'model_X': model_X_smapes,
    # ...
}
smapes_boxplot(smapes, title='sMAPEs on air')

In [ ]:
elapsed_times = {
    'NaiveSeasonal last': naive_seasonal_last_elapsed_time,
    # 'model_X': model_X_elapsed_time,
    # ...
}
elapsed_time_barplot(elapsed_times, title='Duration on air')

### Conclusions so far:
You can also try to directly predict some of the forecasts in order to visualise them.

What are your conclusions so far?

What are your best forecasts? Let us know!

## Part 2: Global models on the `air` dataset
In this section we will use "global models" - that is, models that fit on multiple series at once (more information in [this section of the documentation](https://unit8co.github.io/darts/quickstart/00-quickstart.html#Machine-learning-and-global-models)). Darts has essentially two kinds of global models:
* `RegressionModels` which are wrappers around sklearn-like regression models (Part 2.1).
* PyTorch-based models, which offer various deep learning models (Part 2.2).

Both models can be trained on multiple series by "tabularizing" the data - i.e., taking many (input, output) sub-slices from all the training series, and training machine learning models in a supervised fashion to predict the output based on the input ([more details here](https://unit8co.github.io/darts/userguide/torch_forecasting_models.html#in-depth-look-at-how-input-data-is-used-when-training-and-predicting-with-tfms)).

**Your turn** We will start by defining a function `eval_global_model()` which works similarly to `eval_local_model()`, but on global models. You can complete it below (hint: you will not need the for-loop that was present in `eval_local_model()`).

In [ ]:
def eval_global_model(train_series: List[TimeSeries],
                      test_series: List[TimeSeries],
                      model_cls,
                      **kwargs) -> Tuple[List[float], float]:

    start_time = time.time()

    # build your model here
    ...

    # fit your model here
    ...

    # get some predictions here
    preds = ...

    elapsed_time = time.time() - start_time

    smapes = eval_forecasts(preds, test_series)
    return smapes, elapsed_time

### Part 2.1: Using Darts `RegressionModel`s.
`RegressionModel` in Darts are forecasting models that can wrap around any "scikit-learn compatible" regression model to obtain forecasts. Compared to deep learning, they represent good "go-to" global models because they typically don't have many hyper-parameters and can be faster to train. In addition, Darts also offers some "pre-packaged" regression models such as `LinearRegressionModel` and `LightGBMModel`.

We'll now use our function `eval_global_models()`. In the following cells, you can try using some regression models, for example:
* `LinearRegressionModel`
* `LightGBMModel`
* `RegressionModel`(your_sklearn_model)

You can refer to [the API doc](https://unit8co.github.io/darts/generated_api/darts.models.forecasting.regression_model.html) for how to use them.

Important parameters are `lags` and `output_chunk_length`. They determine respectively the length of the lookback and "lookforward" windows used by the model, and they correspond to the lengths of the input/output subslices used for training. For instance `lags=34` and `output_chunk_length=12` mean that the model will consume the past 34 lags in order to predict the next 12. In our case, because the shortest training series has length 46, we must have `lags + output_chunk_length <= 46`. (Note that `lags` can also be a list of integers representing the individual lags to be consumed by the model instead of the window length).

In [ ]:
# model_X_smapes, model_X_elapsed_time = eval_global_model(air_train, air_test, GlobalModelX, **hyper_params_for_model_X)

### Part 2.2: Using deep learning
Below, we will train an N-BEATS model on our `air` dataset. Again, you can refer to [the API doc](https://unit8co.github.io/darts/generated_api/darts.models.forecasting.nbeats.html) for documentation on the hyper-parameters.
The following hyper-parameters should be a good starting point, and training should take in the order of a minute or two.

During training, you can have a look at the [N-BEATS paper](https://arxiv.org/abs/1905.10437).

In [ ]:
# Slicing hyper-params:
IN_LEN = 24
OUT_LEN = 12

# Architecture hyper-params:
NUM_STACKS = 18
NUM_BLOCKS = 3
NUM_LAYERS = 3
LAYER_WIDTH = 180
COEFFS_DIM = 6
LOSS_FN = SmapeLoss()

# Training settings:
LR = 5e-4
BATCH_SIZE = 1024
NUM_EPOCHS = 4

You can now complete the skeleton below to build, train and predict using an N-BEATS model:

In [ ]:
# reproducibility
np.random.seed(42)
torch.manual_seed(42)

## Use this to specify "optimizer_kwargs" parameter of the N-BEATS model:
optimizer_kwargs={'lr': LR},

## In addition, when using a GPU, you should specify this for
## the "pl_trainer_kwargs" parameter of the N-BEATS model:
pl_trainer_kwargs={"enable_progress_bar": True,
                   "accelerator": "gpu",
                   "gpus": -1,
                   "auto_select_gpus": True}

start_time = time.time()

nbeats_model_air = ... # Build the N-BEATS model here

nbeats_model_air.fit(..., # fill in series to train on
                     ...) # fill in number of epochs

# get predictions
nb_preds = ...

nbeats_smapes = eval_forecasts(nb_preds, air_test)
nbeats_elapsed_time = time.time() - start_time

In [ ]:
smapes_2 = smapes | {
    # ... Fill in here sMAPEs values of any global model you tried
    'NBeats': nbeats_smapes,
  }

smapes_boxplot(smapes_2, title='sMAPEs on air')

In [ ]:
elapsed_time_2 = elapsed_times | {
    # ... Fill in here duration values of any global model you tried
    'NBeats': nbeats_elapsed_time,
}
elapsed_time_barplot(elapsed_time_2, title='Durations on air')

What are your conclusions so far, and which results did you manage to get (let us know!)

## Part 3: Training an N-BEATS model on `m4` dataset and use it to forecast `air` dataset
Deep learning models often do better when trained on *large* datasets. Let's try to load all 48,000 monthly time series in the M4 dataset and train our model once more.

In [ ]:
m4_train, m4_test = load_m4()

# filter to keep only those that are long enough
filtered = filter(lambda t: len(t[0]) >= 48, zip(m4_train, m4_test))
m4_train, m4_test = zip(*filtered)
m4_train, m4_test = list(m4_train), list(m4_test)

print('There are {} series of length >= 48.'.format(len(m4_train)))

We can start from the same hyper-parameters as before.

With 48,000 M4 training series being on average ~200 time steps long, we would end up with ~10M training samples. With such a number of training samples, each epoch would take too long. So here, we'll limit the number of training samples used per series. This is done when calling `fit()` with the parameter `max_samples_per_ts`. We add a new hyper-parameter `MAX_SAMPLES_PER_TS` to capture this.

Since the M4 training series are all >= 48 time steps long, we can also use a slightly longer `input_chunk_length` of 36.

In [ ]:
# Slicing hyper-params:
IN_LEN = 36
OUT_LEN = 12

# Architecture hyper-params:
NUM_STACKS = 18
NUM_BLOCKS = 3
NUM_LAYERS = 3
LAYER_WIDTH = 180
COEFFS_DIM = 6
LOSS_FN = SmapeLoss()

# Training settings:
LR = 5e-4
BATCH_SIZE = 1024
MAX_SAMPLES_PER_TS = 8   # <-- new param, limiting nr of training samples per epoch
NUM_EPOCHS = 7

In [ ]:
# reproducibility
np.random.seed(42)
torch.manual_seed(42)

nbeats_model_m4 = NBEATSModel(..., # fill in hyper-params

                              # Don't forget to set the "pl_trainer_kwargs" parameter
                              # if your notebook has a GPU
                              pl_trainer_kwargs=...
                              )

# Train
nbeats_model_m4.fit(..., # fill in series to train on
                    ..., # fill in number of epochs
                    ...) # fill in max number of samples per time series

In [ ]:
## /!\ RUNNING THIS CELL WILL DOWNLOAD AND OVERWRITE THE MODEL nbeats_model_m4

# # Load already trained model
# # !curl -L https://github.com/unit8co/sds2023-forecasting-and-meta-learning/blob/main/data/nbeats_pretrained_model_m4.zip\?raw\=true -o nbeats_pretrained_model_m4.zip
# with zipfile.ZipFile("nbeats_pretrained_model_m4.zip","r") as zip_ref:
#     zip_ref.extractall("/content/")
# nbeats_model_m4 = NBEATSModel.load("nbeats/NBEATSModel.pt")

We can now use our M4-trained model to get forecasts for the air passengers series. As we use the model in a "meta learning" (or transfer learning) way here, we will be timing only the inference part.

In [ ]:
start_time = time.time()
preds = ... # get forecasts
nbeats_m4_smapes = eval_forecasts(preds, air_test)
nbeats_m4_elapsed_time = time.time() - start_time

What are your conclusions?

### Try training other global models on `m4` and applying on airline passengers
You can now try to train other global models on the M4 dataset in order to see if we can get similar results. If that's taking too long, it might be a good idea to take only e.g., 5'000 or 10'000 time series. You can do this easily by training on, say, `random.choices(m4_train, k=5000)` instead of `m4_train`. You will again need to specify some small enough value for `max_samples_per_ts` in order to limit the number of training samples.

In [ ]:
# reproducibility
random.seed(42)

# model_X = GlobalModelX(...)
# model_X.fit(...,
#             max_samples_per_ts=...)

In [ ]:
# start_time = time.time()
# preds = ...  # Get predictions
# model_X_smapes = eval_forecasts(preds, air_test)  # compute errors
# model_X_elapsed_time = time.time() - start_time  # store timing

In [ ]:
smapes_3 = smapes_2 | {
    'NBeats M4': nbeats_m4_smapes,
    # 'model_X': model_X_smapes
    # ...
}
smapes_boxplot(smapes_3, title='sMAPEs on air')

elapsed_time_3 = elapsed_time_2 | {
    'NBeats M4': nbeats_m4_elapsed_time,
    # 'model_X': model_X_elapsed_time
    # ...
}
elapsed_time_barplot(elapsed_time_3, title='Durations on air')

## Part 4: Forecasting the `m3` dataset

Until now, we have seen that we can use some M4-trained models to predict another dataset, namely the `air` dataset.
But can we try to convince ourselves a bit more, and try the same approach on a third dataset?

In this part of the notebook, we propose to consolidate all our learnings so far using the `m3` dataset:
* Try fitting local models directly on `m3`
* Try fitting global ML models directly on `m3` --> how far can you push it?
* Try applying our previous M4-trained model on `m3` --> what are your conclusions?

Hint: The Theta model was one of the best performing model during the M3 competition.

In [ ]:
# First, load the actual dataset
m3_train, m3_test = load_m3()

In [ ]:
# Then try your models :)